# CRM using Simpy

CRM system simulation using simpy. 

Company CRM populated with accounts (customer companies) and opportunities coming from time to time from these accounts and which can be bidded, won or lost by Company.

Principles: Account "agents" wich will go through a set of events, namely:
- inbound_mktg_event: account finds the contant and follows a CTA to be added into Company MQL list
- outbount_mkt_event: account reached by Company outbound marketing actions and follows a CTA to be added into Company MQL
- inbound_sales_event: account is identified as a sales qualified leads and converted into prospect
- outbound_sales_event: account reached out by Company sales representative and accepts call to become a prospect
and other similar events representing how an account moves from stage to stage through the sales funnel and sales process

Each event is the result of:
- the current phase where the account is 
- an adjustable random variable, using some paramaters (example, conversion rate for CTA, ...)

In [23]:
import random
import simpy

from eccore.ipython import nb_setup
nb_setup()

from agents import Account, account_arrival, accounts_created_before, periodic_reporter
from agents import LEAD_CONVERSION_RATES, DELAY_RANGES

from enums import AccountStage, AccountType

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Set autoreload mode


In [24]:
random.seed(1988)
env = simpy.Environment()
env.accounts = []
Account.cnt = 0
initial_accounts = 5
arrival_rate = 0.001  # on average, one new account every 2 units of time
env.process(account_arrival(env, initial_accounts, arrival_rate))
env.process(periodic_reporter(env, 2))
env.run(until=50)

[0.00] Acct 6: Account created: Acct 6 (UID: 9bf92f00-5cbd-49d8-951b-f2cd092c5052) at stage LEAD
[0.00] Acct 7: Account created: Acct 7 (UID: 697c6f6b-e46c-4f40-8e4a-7a72749f93b7) at stage LEAD
[0.00] Acct 8: Account created: Acct 8 (UID: 7e1e6568-b299-4aed-9071-e25bef6aabef) at stage LEAD
[0.00] Acct 9: Account created: Acct 9 (UID: da57811c-81da-4bee-b5ca-5356c3d1dcd2) at stage LEAD
[0.00] Acct 10: Account created: Acct 10 (UID: 237e1134-41aa-4cfc-a1a1-54456e29e5cc) at stage LEAD
[0.00] Acct 6: entering run for Acct 6, LEAD
[0.00] Acct 6: Lead is converting to MQL via inbound marketing event
[0.00] Acct 6: entering run for Acct 6, LEAD
[0.00] Acct 6: Lead is converting to MQL via inbound marketing event
[0.00] Acct 7: entering run for Acct 7, LEAD
[0.00] Acct 7: Lead is converting to SQL via outbound sales event
[0.00] Acct 7: entering run for Acct 7, LEAD
[0.00] Acct 7: Lead is converting to MQL via inbound marketing event
[0.00] Acct 8: entering run for Acct 8, LEAD
[0.00] Acct 8: 

In [25]:
[a.name for a in env.accounts]

['Acct 6', 'Acct 7', 'Acct 8', 'Acct 9', 'Acct 10']

In [26]:
accounts_created_before(env=env, t=18)

# Sandbox

## Concept of inbox (store)

In [27]:
class Account:
    def __init__(self, env, name):
        self.env = env
        self.name = name
        self.inbox = simpy.Store(env)
        self.env.accounts.append(self)

    def run(self):
        while True:
            print(f"{self.env.now}: {self.name} waiting for messages...")
            msg = yield self.inbox.get()
            print(f"{self.env.now}: {self.name} received {msg}")

def mktg_agent(env, account_store, delay=5):
    yield env.timeout(delay)
    yield account_store.put("marketing outreach")
    print(f"{env.now}: Agent sent event to account")


In [28]:
def report(env):
    print(f"\n--- Report at time {env.now} ---")
    for account in env.accounts:
        print(f"{account.name}: {account.inbox.items}")
    print("-------------------------------\n")

# ... Account class and other functions ...

def periodic_reporter(env, interval):
    # First report at time 1
    yield env.timeout(1)
    report(env)
    while True:
        yield env.timeout(interval)
        report(env)

In [29]:
env = simpy.Environment()
env.accounts = []  # type: ignore


account1 = Account(env, "A1")
account2 = Account(env, "A2")

env.process(account1.run())
env.process(account2.run())

env.process(mktg_agent(env, account1.inbox, 3))
env.process(mktg_agent(env, account2.inbox))
# env.process(periodic_reporter(env, 1))
# env.run(until=10)

<Process(mktg_agent) object at 0x7f5fdd4c9f00>

In [30]:
print(env.peek())
env.run(env.peek()+1)

0
0: A1 waiting for messages...
0: A2 waiting for messages...


In [31]:
print(env.peek())
env.run(env.peek()+1)

3
3: Agent sent event to account
3: A1 received marketing outreach
3: A1 waiting for messages...


In [32]:
print(env.peek())
env.run(env.peek()+1)

5
5: Agent sent event to account
5: A2 received marketing outreach
5: A2 waiting for messages...


In [33]:
print(env.peek())
env.run(env.peek()+1)

inf
